In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset 
import evaluate

In [ ]:
#trust_remote_code,代表信任遠端資料,如不寫會有信任與否的提示
ner_datasets = load_dataset("peoples_daily_ner", cache_dir='./data', trust_remote_code=True)
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [8]:
ner_datasets['train'][0]

# id,tokens,ner_tags

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [ ]:
ner_datasets['train'].features 

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [12]:
#取得label_list
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [15]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [18]:
#不正確,這是以字為單位,而不是以句子為單位
from pprint import pprint
pprint(tokenizer(ner_datasets['train'][0]['tokens']))

{'attention_mask': [[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]],
 'input_ids': [[101, 3862, 102],
               [101, 7157, 102],
               [101, 3683, 102],
               [101, 6612, 102],
               [101, 1765, 102],
               [101, 4157, 102],
               [101, 1762, 102],
               [101, 1336, 102],
               [101, 7305, 102],
               [101, 680, 102],
               [101, 7032, 102],
               [101, 7305, 102],
               [101, 722, 102],
              

In [19]:
pprint(tokenizer(ner_datasets['train'][0]['tokens'],is_split_into_words=True))


{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101,
               3862,
               7157,
               3683,
               6612,
               1765,
               4157,
               1762,
               1336,
               7305,
               680,
               7032,
               7305,
               722,
               7313,
               4638,
               3862,
               1818,
               511,
               102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
#文字字數8個加上101,102共10個
#input_ids = 101+字數+102,共10個
#所以labels也必需要10個
pprint(tokenizer(['小','明','愛','去','海','邊','釣','魚'],is_split_into_words=True))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 2207, 3209, 2695, 1343, 3862, 6920, 7037, 7797, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [23]:
#中英文混合
#文字字數8個加上101,102共10個
#input_ids = 101+10個字數+102,共12個 -> 英文字有sub_word的關係
#所以labels必需要12個
pprint(tokenizer(['小','明','interest','to','海','邊','釣','魚'],is_split_into_words=True))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101,
               2207,
               3209,
               10673,
               12865,
               8415,
               8228,
               3862,
               6920,
               7037,
               7797,
               102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
pprint(tokenizer(['小','明','interest','to','海','邊','釣','魚'],is_split_into_words=True))
ner_tags = [1, 2, 0, 0, 0, 0, 0, 0]
#input_ids = [101,2207,3209,10673,12865,8415,8228,3862,6920,7037,7797,102]
#101,102,要產生-100
#我們預計要產生的labels必需是=[-100,1,2,0,0,0,0,0,0,0,0,-100],共12,滿足input_ids的數量,但ner_tags欄位不可以滿足
#使用word_ids()解決問題
res = tokenizer(['小','明','interest','to','海','邊','釣','魚'],is_split_into_words=True)
print(res.word_ids())
#[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, None]
#None -> 101, 102
#0 -> '小'
#1 -> '明'
#2,2,2 -> 'interest'
#3 -> 'to'
#4 -> '海'
#5 -> '邊'
#6 -> '釣'
#7 -> '魚'

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101,
               2207,
               3209,
               10673,
               12865,
               8415,
               8228,
               3862,
               6920,
               7037,
               7797,
               102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, None]


In [50]:
ner_tags = [1, 2, 0, 0, 0, 0, 0, 0] #自已手動的
res = tokenizer(['小','明','interest','to','海','邊','釣','魚'],max_length=128,truncation=True,is_split_into_words=True)
word_id = res.word_ids()
label_id = []
for id in word_id:
    if id is None:
        label_id.append(-100)
    else:
        #print(id)
        label_id.append(ner_tags[id])
print(label_id)

[-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
#1筆1筆處理的方法
def process_one(example):
    ner_tags = example['ner_tags']
    res = tokenizer(example['tokens'],max_length=128, truncation=True, is_split_into_words=True)
    word_id = res.word_ids()
    label_id = []
    for id in word_id:
        if id is None:
            label_id.append(-100)
        else:
            label_id.append(ner_tags[id])
    res['labels'] = label_id

    return res
tokenized_datasets = ner_datasets.map(process_one)
print(tokenized_datasets)
pprint(tokenized_datasets['train'][0]['labels'])

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})
[-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]


In [55]:
#一個批次處理的方式
from pprint import pprint
def process_batch(examples):
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels

    return tokenized_examples

tokenized_datasets = ner_datasets.map(process_batch,batched=True)
print(tokenized_datasets)
print(tokenized_datasets['train'][0]['labels'])

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})
[-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]


In [56]:
#label_list
#['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=len(label_list)) #預設是2個,現在是7個

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
#查詢模型的num_labels
model.num_labels

7

In [58]:
seqeval = evaluate.load('seqeval')
seqeval #查看所需要的說明書

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [59]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    #print(predictions),評估時print()可以比較了解
    predictions = np.argmax(predictions, axis=-1) #變為和label一樣的一維資料

    #刪除-100
    truth_predictions = [
        [label_list[p] for p,l in zip(prediction, label) if p != -100]
        for prediction, label in zip(predictions, labels)
    ]

    truth_labels = [
        [label_list[l] for p,l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=truth_predictions, references=truth_labels, mode='strict', scheme='IOB2')
    return{
        "f1": result['overall_f1']
    }

In [61]:
args = TrainingArguments(
    output_dir = "models_for_ner",
    per_device_eval_batch_size=64,
    per_device_train_batch_size=128,
    eval_strategy='epoch',
    save_strategy='epoch',
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    logging_steps = 50,
    num_train_epochs=3,
    report_to='none'
)

In [63]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

In [ ]:
trainer.train()